# Geometry Optimization using VASP and ASE

Author: PhD. Hugo de Lacerda Coutinho Neto

Contact: [hneto@peq.coppe.ufrj.br](mailto:hneto@peq.coppe.ufrj.br) - [Programa de Engenharia Química, PEQ/COPPE, UFRJ, Brasil](https://www.peq.coppe.ufrj.br/)

Cod. Ref.: [Prof. Elvis do A. Soares](https://github.com/elvissoares) 

---

Importando variáveis do VASP

# Importing libraries

In [33]:
import os
# Definindo o path para os arquivos de potencial de pseudopotenciais do VASP
# Certifique-se de que o caminho esteja correto para o seu sistema
os.environ['VASP_PP_PATH'] = '/home/hugo/Documents/Programs/VASP/vasp-6.5.1/pp'
os.environ['ASE_VASP_COMMAND'] = 'mpirun -np 1 vasp_std_gpu'
os.environ['NO_STOP_MESSAGE'] = '1' # to avoid warning from mpirun

In [37]:
import numpy as np
import matplotlib.pyplot as plt

from ase import Atoms, Atom
from ase.build import molecule
from ase.io import write, read
from ase.calculators.vasp import Vasp      # Importando o VASP calculator do ASE

from ase.visualize.plot import plot_atoms
from ase.visualize import view

from ase.constraints import FixAtoms, FixCartesian
from ase.geometry import cellpar_to_cell   # Convert cell parameters to cell vectors

from ase.vibrations import Vibrations

# Geometry Optimization of Cellulose Crystal

In [ ]:
ActualDir   = os.getcwd()

## Optmize Geometry (Only Hydrogen is Free)

### I/O Definitions

In [39]:
# *************** OUT DIR ***************
InputDir   = "hydrogen_bonds"
Cif_name   = "Cellulose_Beta.cif"
InputFile  = os.path.join(ActualDir, InputDir, Cif_name)

Defines the output dir

In [40]:
# *************** OUT DIR ***************
OutputDir0   = "hydrogen_bonds/MonobDGlu_MonobDGlu_H_variable"
OutputDir0   = os.path.join(ActualDir, OutputDir0)

Import .cif of Cellulose Beta

Crystal Structure and Hydrogen-Bonding System in Cellulose I Beta‚ from Synchrotron X-ray and Neutron Fiber Diffraction

(2002) Yoshiharu Nishiyama, Paul Langan, and Henri Chanzy

Link:

http://openmopac.net/PM7_accuracy/data_solids/Cellulose_1_beta__ja0257319_s1__Jmol.html

 226 Cellulose 1 beta (ja0257319 s1)

(Previous)    Cellulose 1 alpha (ja037055wsi20030711 112804)

(Back)          Elements: C 6 H 10 O 5 (Z = 4)     (Periodic Table)

(Next)          alpha-Chitin (SUSGIW)

Reference: "Crystal Structure and Hydrogen-Bonding System in Cellulose I beta from Synchrotron X-ray and Neutron Fiber Diffraction"
Yoshiharu Nishiyama, Paul Langan, and Henri Chanzy, Journal of the American Chemical Society 2002 124 (31), 9074-9082
Carbohydrates: Glucose, beta-Maltose monohydrate, Sucrose,
alpha-Lactose monohydrate, beta-D-Fructose, beta-D-Galactose,
beta-Cellobiose, Cellulose 1 alpha, Cellulose 1 beta.

                      Unit Cell Parameters:       a      b      c   alpha   beta  gamma   Volume  Density       Heat of Formation (Kcal/mol)
                                     X-ray:    10.38   8.20   7.78  96.55  90.00  90.00    658.26  1.636         -224.2 calc'd using PM7
                                       PM7:    10.27   8.88   7.39  93.11  90.10  99.95    662.77  1.625         -236.0 calc'd using PM7
                                       PM6:    10.43   7.95   7.39  93.22  86.91  95.53    607.94  1.772         -230.0 calc'd using PM6


See the supercell (how is the cellulose polymer)

In [16]:
Cellulose_Beta = read(InputFile)

See the unit cell

In [17]:
print(Cellulose_Beta)
view(Cellulose_Beta, viewer='x3d')

Atoms(symbols='C24H48O20', pbc=True, cell=[[7.784, 0.0, 0.0], [-0.935489919859435, 8.147469521872505, 0.0], [0.0, 0.0, 10.38]], masses=..., spacegroup_kinds=...)


### Geometry Optimization Definition and Execution

Contrain to maintain carbon and oxygen fixed

In [19]:
mask = [atom.symbol in ('C','O') for atom in Cellulose_Beta]  # True for C, O
Cellulose_Beta.set_constraint(FixAtoms(mask=mask))

In [ ]:
# (paper) dx.doi.org/10.1021/jp205827

calc = Vasp(
    directory=OutputDir0,
    xc='PBE',
    encut=800,          # Energy cutoff for plane waves (from paper)
    kpts=[2,2,2],       # Reciprocal space sampling (from paper)
    gamma=True,
    ismear=0, 
    sigma=0.05,            
    ediff=1e-7,         # SCF convergence criterion (from paper)
    ediffg=-0.02,       # Break condition for ionic steps (negative value means force) (from paper)
    ibrion=2,           # IBRION=2 conjugate gradient (from paper)
    isif=2,             # Cell Shape and Volume Fixed 
    nsw=350,            # Maximum number of ionic steps
    lreal='Auto',       # Projection operators are evaluated in real space if the unit cell is large enough
    ivdw=12,            # DFT-D3 method with Becke-Johnson damping function.[3] Available as of VASP.5.3.4. 
    lwave=True, 
    lcharg=True, 
    lvtot=True,
    #ldipol=True, idipol=3,
    atoms=Cellulose_Beta
)

In [25]:
E = Cellulose_Beta.get_potential_energy()
print(f"Relaxed energy: {E:.6f} eV")

Relaxed energy: -525.292320 eV


When you run get_potential_energy(), ASE checks the target directory.

If it finds a valid VASP calculation that is incomplete, it will automatically restart it from the last c
completed step.

In [26]:
write('Cellulose_Beta_H_relaxed.pdb', Cellulose_Beta)

## Frequency Calculations

### Frequency Data with lowest comp. cost

#### I/O Definition

In [55]:
# Read the structure with relaxed H atoms
ActualDir   = os.getcwd()

OutputDir0   = "hydrogen_bonds/MonobDGlu_MonobDGlu_H_variable"
OutputDir0   = os.path.join(ActualDir, OutputDir0)

OutputDir_contcar = os.path.join(OutputDir0, 'CONTCAR')
relaxed_cellulose = read(OutputDir_contcar)

OutputDir1        = "hydrogen_bonds/MonobDGlu_frequency_Simple"
OutputDir1        = os.path.join(ActualDir, OutputDir1)

#### Simulation Environment Definition

In [56]:
# IMPORTANT: Remove the constraints for the frequency calculation
relaxed_cellulose.set_constraint(None)

# (paper) 10.1021/acs.jpcb.5b08015
calc_frequency = Vasp(
    directory=OutputDir1,
    xc='PBE',       # Exchange-correlation functional (from paper)
    encut=800,      # Kinetic energy cutoff for plane waves (from paper)
    kpts=(2, 2, 2), # Reciprocal space sampling (from paper)
    #kpts=(1, 1, 1), # Reciprocal space sampling (from paper)
    gamma=True,     # Use Gamma-centered k-point grid
    ismear=0,       # Smearing method (0 = Gaussian)
    sigma=0.05,     # Smearing width (eV)
    # See that: https://vasp.at/wiki/Smearing_technique
    # The Gaussian-smearing method leads to very reasonable results in most cases. 
    # Within this method it is necessary to extrapolate from finite SIGMA results to SIGMA = 0 results. 
    # You can find an extra line in the OUTCAR file: energy( SIGMA→0 ), giving the extrapolated results. 
    # However, this value will not be accurate without systematically reducing SIGMA. 
    # Typically, you will need to use a SIGMA of 0.03 to 0.1 for converged results. 
    ediff=1e-7,     # Self Consistent Loop parameter convergence
    #ediff=1e-1,     # Self Consistent Loop parameter convergence
    ibrion=5,       # Use IBRION=5 for Hessian matrix calculation (finite differences)
    nsw=1,          # Only one step is needed for a frequency calculation
    nfree=2,        # Use two-sided differences for higher accuracy (from paper)
    potim=0.015,    # Displacement distance (Å) (from paper)
    lreal='Auto',   # Projection operators in "automatic" determination space
    ivdw=12,        # Dispersion correction
    lwave=True,
    lcharg=True
)

# ASSIGN the calculator to the atoms object
relaxed_cellulose.calc = calc_frequency

#### Generate Frequency Data

In [57]:
# This will run the expensive frequency calculation
energy_with_frequencies = relaxed_cellulose.get_potential_energy()

In [58]:
print(f"New relaxed energy: {energy_with_frequencies:.6f} eV")

New relaxed energy: -525.282854 eV


In [59]:
# The frequencies are now in the OUTCAR. You can use a tool to read them.
# For a simple list, you can sometimes use:

from ase.io.vasp import read_vasp_out

data        = read_vasp_out(os.path.join(OutputDir1, 'OUTCAR'))
frequencies = data.get_vibrations().get_frequencies()

print(frequencies)

AttributeError: 'Atoms' object has no attribute 'get_vibrations'

Step 7: (Critical) Scale the Frequencies

As noted in the paper, DFT-calculated frequencies, especially for OH stretches, are not perfect. You must scale them to match known experimental values. The paper uses the alkyl (C-H) stretch and bend modes as references because they are well-known and less coupled.

    Identify the calculated C-H stretch frequencies (~2900-3000 cm⁻¹).

    Calculate a scaling factor: scaling_factor = experimental_CH_frequency / calculated_CH_frequency.

    Apply this same scaling factor to all your calculated OH frequencies.

For example, if a known C-H peak is at 2890 cm⁻¹ and you calculate it at 2750 cm⁻¹, your scaling factor is 2890 / 2750 ≈ 1.05. You would then multiply all your OH frequencies by 1.05.

### Frequency Data with highest comp. cost

#### I/O Definition

In [66]:
# Read the structure with relaxed H atoms
ActualDir   = os.getcwd()

OutputDir0   = "hydrogen_bonds/MonobDGlu_MonobDGlu_H_variable"
OutputDir0   = os.path.join(ActualDir, OutputDir0)

OutputDir_contcar = os.path.join(OutputDir0, 'CONTCAR')
relaxed_cellulose = read(OutputDir_contcar)

OutputDir2        = "hydrogen_bonds/MonobDGlu_frequency_Complex"
OutputDir2        = os.path.join(ActualDir, OutputDir2)

#### Simulation Environment Definition

In [ ]:
# 1. First, run a single-point calculation to get the ground state
calc_sp = Vasp(
    directory=OutputDir2,
    xc='PBE',
    encut=800,
    kpts=(2, 2, 2),
    gamma=True,
    ismear=0,
    sigma=0.05,
    ediff=1e-7,
    ibrion=-1,  # Important: No ionic relaxation
    nsw=0,      # Important: No ionic steps
    lreal='Auto',
    ivdw=12,
    lwave=True,
    lcharg=True,
)
relaxed_cellulose.calc = calc_sp

In [68]:
relaxed_cellulose.get_potential_energy() # Get converged charge density

-525.29231988

In [ ]:
# 2. NOW use the Vibrations class
vib = Vibrations(relaxed_cellulose)
vib.run() # This will create many displacements and run calculations for each

In [ ]:
print(vib.summary())